# Document Analysis in Python
In this notebook we will cover:
- Reading document data into memory
- Creating bag of words features
- Creating smoothed tf-idf features

In [1]:
import requests
import json
from contextlib import closing

# get API key saved on hardrive
with open('../NYTimesAPI.txt') as f:
    api_key = f.read() # read in my private key (sorry, not in this repo ¯\_(ツ)_/¯ )
    
# make base URL and dictionary of get request key/values
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json"
payload = {'api-key': api_key, 'q':'Olympics'} # key/values for get request (look up in api, there are lots)

# Perform the actual request]
with closing(requests.get(url,params=payload))as r:
    articles = r.json()
    print(articles)

{'response': {'docs': [{'subsection_name': 'Olympics', 'lead_paragraph': 'When the German twins Anna and Lisa Hahner joined hands as they finished deep in the pack in the women’s marathon, they quickly drew sharp criticism from their federation.', 'section_name': 'Sports', 'snippet': 'When the German twins Anna and Lisa Hahner joined hands as they finished deep in the pack in the women’s marathon, they quickly drew sharp criticism from their federation.', 'source': 'The New York Times', 'pub_date': '2016-08-17T00:00:00Z', 'blog': [], 'slideshow_credits': None, 'byline': {'original': 'By CHRISTOPHER CLAREY', 'person': [{'organization': '', 'role': 'reported', 'firstname': 'Christopher', 'lastname': 'CLAREY', 'rank': 1}]}, 'document_type': 'article', 'headline': {'print_headline': 'Hand in Hand: Did Their Finish Cross a Line?', 'content_kicker': 'On Olympics', 'main': 'Twins Finish Marathon Hand in Hand, but Their Country Says They Crossed a Line', 'kicker': 'On Olympics'}, 'word_count':

In [ ]:
# OR
# could use the NYTIMES API like this
from nytimesarticle import articleAPI

with open('../NYTimesAPI.txt') as f:
    api_key=f.read() # read in my private key (not in this repository, sorry)
    
api = articleAPI(api_key)

articles = api.search(start_date='20160601')
articles

In [ ]:
# OR we can load an example query like this
# run this block of code if you can't run anything else
import json 
with open('data/nytime.json') as f:
    articles = json.loads(f.read())
    
articles

In [2]:
# get the summary text from each article
summary_text = [x['lead_paragraph'] for x in articles['response']['docs']]
summary_text

['When the German twins Anna and Lisa Hahner joined hands as they finished deep in the pack in the women’s marathon, they quickly drew sharp criticism from their federation.',
 'David Goldblatt’s “The Games” recalls unflattering aspects of the Olympics long before doping and gender testing.',
 '“What was in my mind was, I had to get a gold medal,” said Shaunae Miller, who won the 400 in spectacular fashion, depriving Allyson Felix of career-capping satisfaction.',
 'Had the United States women competed as a separate country, they would have been third in the overall medal chart with 61.',
 'Naturally high testosterone levels in women have not been scientifically demonstrated to give them a more significant edge than many other factors.',
 'Penalizing a country for doping seems to be a much more effective way to ensure rapid and enduring change.',
 'With some of the smallest crowds in Olympic history at Rio, Coe, president of the I.A.A.F., is looking afresh at ways to save the sport.',


# Converting document data into different representations
First lets go through and count the unique words in each opening sentence (that is what the NYTimes give us for free).
- http://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

In [3]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer() # an object capable of counting words in a document!

# count_vect.fit(summary_text)
# count_vect.transform(summary_text)
bag_words = count_vect.fit_transform(summary_text)

In [4]:
print(bag_words.shape) # this is a sparse matrix
print(bag_words[0])

(10, 167)
  (0, 159)	1
  (0, 141)	3
  (0, 61)	1
  (0, 148)	1
  (0, 12)	1
  (0, 11)	1
  (0, 84)	1
  (0, 69)	1
  (0, 79)	1
  (0, 70)	1
  (0, 13)	1
  (0, 145)	2
  (0, 55)	1
  (0, 36)	1
  (0, 76)	2
  (0, 107)	1
  (0, 163)	1
  (0, 90)	1
  (0, 113)	1
  (0, 41)	1
  (0, 127)	1
  (0, 32)	1
  (0, 58)	1
  (0, 142)	1
  (0, 53)	1


# Self Test: ML01b.3: 
Do you expect the vocabulary from the articles above to be:
- A. Greater than 1M words
- B. Greater than 10,000 words
- C. Fewer than 10,000 words

In [6]:
print(len(count_vect.vocabulary_))
print(count_vect.vocabulary_)

167
{'separate': 126, 'them': 143, 'way': 155, 'save': 122, 'drug': 42, 'particular': 108, 'hands': 70, 'rival': 119, 'make': 88, 'united': 150, 'to': 147, 'german': 61, 'has': 71, 'greatness': 67, 'is': 77, 'drew': 41, 'overall': 106, 'an': 10, 'change': 26, 'team': 137, 'rio': 118, 'got': 66, 'hahner': 69, 'and': 11, 'medal': 92, 'ensure': 47, 'displaced': 39, 'recalls': 116, 'third': 146, 'demonstrated': 37, 'games': 59, 'with': 162, 'had': 68, '400': 3, 'as': 13, 'power': 110, 'federation': 53, 'more': 95, 'athletes': 16, 'marathon': 90, 'fans': 51, 'it': 78, 'eaton': 43, 'less': 81, 'goldblatt': 65, 'miller': 93, 'bolt': 21, 'their': 142, 'penalizing': 109, 'from': 58, '10': 0, 'they': 145, 'usain': 152, 'effective': 45, 'give': 63, 'significant': 129, 'may': 91, 'high': 73, 'allyson': 8, 'than': 140, 'up': 151, 'felix': 54, 'finished': 55, 'capping': 24, 'have': 72, 'sport': 133, 'much': 96, 'straight': 136, 'wink': 161, 'weight': 157, 'naturally': 98, 'not': 101, 'country': 31, 

In [7]:
# we can still look at the data using an inverse transform
count_vect.inverse_transform(bag_words[0])

[array(['when', 'the', 'german', 'twins', 'anna', 'and', 'lisa', 'hahner',
        'joined', 'hands', 'as', 'they', 'finished', 'deep', 'in', 'pack',
        'women', 'marathon', 'quickly', 'drew', 'sharp', 'criticism',
        'from', 'their', 'federation'], 
       dtype='<U14')]

In [8]:
# now let's create a pandas API out of this
import pandas as pd

pd.options.display.max_columns = 999
df = pd.DataFrame(data=bag_words.toarray(),columns=count_vect.get_feature_names())

In [9]:
df # display the full bag of words matrix  

,10,1994,1995,400,61,about,achievement,afresh,allyson,also,an,and,anna,as,aspects,at,athletes,attention,be,been,before,bolt,but,by,capping,career,change,chart,city,coe,competed,country,criticism,crowds,david,decathlon,deep,demonstrated,depriving,displaced,doping,drew,drug,eaton,edge,effective,enduring,ensure,entertained,even,factors,fans,fashion,federation,felix,finished,for,free,from,games,gender,german,get,give,gold,goldblatt,got,greatness,had,hahner,hands,has,have,high,history,if,in,is,it,joined,leader,less,levels,lifting,lisa,long,looking,loroupe,make,many,marathon,may,medal,miller,mind,more,much,my,naturally,new,nod,not,of,olympic,olympics,other,overall,pack,particular,penalizing,power,pragmatism,president,quickly,rapid,raw,recalls,refugee,rio,rival,said,satisfaction,save,scientifically,second,seems,separate,sharp,shaunae,significant,smallest,some,spectacular,sport,sprint,states,straight,team,testing,testosterone,than,the,their,them,there,they,third,to,twins,unflattering,united,up,usain,want,was,way,ways,weight,what,when,who,wink,with,women,won,would,york
0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,2,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,3,1,0,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,2,0,0,0,1,0,1,0,0,0,1,0,0
3,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,2,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,1,0,1,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
5,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,3,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
7,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,0,0,0,0,1,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,1,0,0,0,1,0,2,0,0,0,0,1,0,2,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0

In [10]:
# print out 10 most common words in our data
df.sum().sort_values()[-10:]

been      3
medal     3
be        3
was       3
they      4
of        5
and       6
to        8
in       10
the      16
dtype: int64

In [ ]:
# print out 10 least common words in our data
df.sum().sort_values()[:10] # small sample size means most words occur one time

# TF-IDF Conversion
We have a very small sample of data, but lets covert to tf-idf for the sake of programming it. Recall that Tf-idf transformation (default in `sklearn` is):

$$ \text{tf}(t,d) = f_{td}\text{, } t\in T \text{ and } d \in D $$

$$ \text{idf}(t,d) = \log{\frac{|D|}{|n_t|}}\text{, where } n_t=d\in D \text{ with } t\in d $$

$$\text{tf-idf}(t,d)=\text{tf}(t,d) \cdot (1+\text{idf}(t,d))$$

In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vect = TfidfVectorizer() # an object capable of counting words in a document!

tfidf_mat = tfidf_vect.fit_transform(summary_text) # that's it! its converted!!

In [12]:
# convert to pandas to get better idea about the data
df = pd.DataFrame(data=tfidf_mat.toarray(),columns=tfidf_vect.get_feature_names())
df

,10,1994,1995,400,61,about,achievement,afresh,allyson,also,an,and,anna,as,aspects,at,athletes,attention,be,been,before,bolt,but,by,capping,career,change,chart,city,coe,competed,country,criticism,crowds,david,decathlon,deep,demonstrated,depriving,displaced,doping,drew,drug,eaton,edge,effective,enduring,ensure,entertained,even,factors,fans,fashion,federation,felix,finished,for,free,from,games,gender,german,get,give,gold,goldblatt,got,greatness,had,hahner,hands,has,have,high,history,if,in,is,it,joined,leader,less,levels,lifting,lisa,long,looking,loroupe,make,many,marathon,may,medal,miller,mind,more,much,my,naturally,new,nod,not,of,olympic,olympics,other,overall,pack,particular,penalizing,power,pragmatism,president,quickly,rapid,raw,recalls,refugee,rio,rival,said,satisfaction,save,scientifically,second,seems,separate,sharp,shaunae,significant,smallest,some,spectacular,sport,sprint,states,straight,team,testing,testosterone,than,the,their,them,there,they,third,to,twins,unflattering,united,up,usain,want,was,way,ways,weight,what,when,who,wink,with,women,won,would,york
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.117616,0.198067,0.168375,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.198067,0.000000,0.000000,0.000000,0.198067,0.000000,0.000000,0.000000,0.000000,0.198067,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.198067,0.000000,0.198067,0.00000,0.000000,0.198067,0.000000,0.000000,0.198067,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.198067,0.198067,0.000000,0.000000,0.000000,0.000000,0.000000,0.175849,0.000000,0.000000,0.198067,0.000000,0.000000,0.000000,0.000000,0.198067,0.000000,0.000000,0.000000,0.000000,0.000000,0.168375,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.198067,0.000000,0.00000,0.000000,0.000000,0.000000,0.198067,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.198067,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.289648,0.198067,0.000000,0.000000,0.294616,0.000000,0.000000,0.198067,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.198067,0.000000,0.000000,0.000000,0.147308,0.000000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.163533,0.000000,0.000000,0.275391,0.000000,0.000000,0.000000,0.000000,0.000000,0.275391,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.275391,0.000000,0.000000,0.000000,0.000000,0.000000,0.234107,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.275391,0.275391,0.000000,0.000000,0.000000,0.000000,0.275391,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.234107,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.182096,0.000000,0.275391,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.275391,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.275391,0.000000,0.000000,0.268484,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.275391,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.00000

In [13]:
# print out 10 words with max tfidf, normalized by document occurrence
df.max().sort_values()[-10:]

before    0.275391
of        0.278547
to        0.282973
they      0.294616
be        0.304369
who       0.344952
was       0.346431
want      0.358043
the       0.395604
at        0.421257
dtype: float64

# Working with (a bit) more data
What if we do not have the memory to deal with dense matrix representatioan and we need to keep it sparse?


In [14]:
from sklearn.datasets import fetch_20newsgroups
bunch = fetch_20newsgroups(subset='train', shuffle=True, random_state=42)

the `bunch` object returned from sklearn is similar to a python dictionary. We can access different fields of the object with keys.

In [15]:
print(bunch.data[0]) # we should split this up by newlines

From: lerxst@wam.umd.edu (where's my thing)
Subject: WHAT car is this!?
Nntp-Posting-Host: rac3.wam.umd.edu
Organization: University of Maryland, College Park
Lines: 15

 I was wondering if anyone out there could enlighten me on this car I saw
the other day. It was a 2-door sports car, looked to be from the late 60s/
early 70s. It was called a Bricklin. The doors were really small. In addition,
the front bumper was separate from the rest of the body. This is 
all I know. If anyone can tellme a model name, engine specs, years
of production, where this car is made, history, or whatever info you
have on this funky looking car, please e-mail.

Thanks,
- IL
   ---- brought to you by your neighborhood Lerxst ----







In [18]:
import numpy as np
idx = round(np.random.rand()*len(bunch.data))
print("\n".join(bunch.data[idx].split("\n")))

From: imj1@cunixa.cc.columbia.edu (Imad M Jureidini)
Subject: Re: Problems with Toshiba 3401 CDROM
Nntp-Posting-Host: cunixa.cc.columbia.edu
Reply-To: imj1@cunixa.cc.columbia.edu (Imad M Jureidini)
Organization: Columbia University
Lines: 22

In article <1r25nt$oa5@ratatosk.uninett.no> hktth@nho.hydro.com writes:
>In article <1993Apr20.191255.10115@news.columbia.edu>, imj1@cunixa.cc.columbia.edu (Imad M Jureidini) writes:
>|> Hi!
>|> 	I recently purchased the Toshiba 3401 CDROM.  I own an Adaptec 1542B
>|> SCSI card, and I have so far failed to get the CDROM to work under DOS.  It
>
>One of the ASPI-drivers (I think it's the ASPICD) supports a /NORST
>paramter, which means to not reset the SCSI bus when it loads. This
>fixed the problem a friend of mine was having with his adaptec+tosh 
>3401.
>
>Regards,
>
>  -Terje
It worked!!!
Thank you very much!


*******************************************************************************
* imj1@cunixa.cc.columbia.edu			    Imad "Hexabyte" Jur

In [19]:
news_tfidf = tfidf_vect.fit_transform(bunch.data) 

In [20]:
news_tfidf.shape

(11314, 130107)

In [21]:
tfidf_vect.vocabulary_

{'486dx33': 14068,
 'refuses': 100404,
 'flkhf': 55882,
 'chesspieces': 39293,
 '80x86': 20140,
 '80w': 20135,
 'mercury': 81182,
 'subtractive': 111477,
 'comparable': 41359,
 'rolfe': 102840,
 '6_8': 17350,
 'drury': 49138,
 'freqencies': 56866,
 'biochemi': 33423,
 '5j4': 16010,
 'wd40': 123530,
 'sx9w': 112470,
 'disproving': 47859,
 'alcohol': 27284,
 'jrobbins': 70806,
 'dolgo': 48497,
 'jannet': 69498,
 'angular': 28267,
 'cq': 43153,
 'melkumians': 80988,
 'idt114': 65768,
 'ff2lk': 55043,
 'rhetorics': 101829,
 'aytall': 30917,
 'pena': 92832,
 'x6': 125866,
 'rosevc': 102985,
 'litre': 76200,
 'runtimes': 103531,
 'convenant': 42412,
 'uncountably': 118529,
 '10011101': 2444,
 'irresponsibility': 68496,
 'chris_chelios': 39568,
 'brads': 34756,
 'weston': 123864,
 'mahaffey': 78922,
 'paperwork': 91943,
 '3310': 11782,
 'survelliance': 112107,
 'nutek': 88435,
 'tbird': 113627,
 '_8nlbzp': 23273,
 'tkojz': 115328,
 '204742': 8495,
 'consortia': 42125,
 'vnet3': 121947,
 'waht

In [22]:
# create pandas dataframe
vec=news_tfidf.max(axis=0)
df = pd.DataFrame(data=vec.toarray(),columns=tfidf_vect.get_feature_names())

In [23]:
df.max().sort_values()[-10:]

kk          0.870294
db          0.871473
scsi        0.875086
blah        0.879426
donoghue    0.891653
00          0.907726
___         0.908826
25          0.913127
forged      0.940511
ax          0.998314
dtype: float64

In [24]:
# now lets do the transformation with a smaller vocabulary
tfidf_vect = TfidfVectorizer(stop_words='english',
                             max_df=0.01,
                             min_df=4)
news_tfidf = tfidf_vect.fit_transform(bunch.data) 
print(news_tfidf.shape)
vec=news_tfidf.max(axis=0)
df = pd.DataFrame(data=vec.toarray(),columns=tfidf_vect.get_feature_names())
df.max().sort_values()[-10:]

(11314, 28593)


dialix            0.947270
blah              0.952881
ualberta          0.956563
stephanopoulos    0.959843
forged            0.971300
mufti             0.976947
ax                0.999881
meyers            1.000000
slower            1.000000
ucsd              1.000000
dtype: float64

# Using your own vocabulary

In [25]:
# read in scrabble dictionary from file
with open('data/ospd.txt') as f:
    vocab = f.read().split('\n')
    
# now lets do the transformation with a custom vocabulary
tfidf_vect = TfidfVectorizer(vocabulary=vocab)
news_tfidf = tfidf_vect.fit_transform(bunch.data) 
print(news_tfidf.shape)
vec=news_tfidf.max(axis=0)
df = pd.DataFrame(data=vec.toarray(),columns=tfidf_vect.get_feature_names())
df.max().sort_values()[-10:]

(11314, 79340)


incoming    0.925305
siemens     0.927114
water       0.928029
echo        0.947289
blah        0.951962
dos         0.953675
lib         0.954914
forged      0.978762
la          0.982737
ax          0.999999
dtype: float64